# Выгрузка данных для анализа уровня лояльности клиентов телекоммуникационной компании

## Импотрируем библиотеки

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

## Подключаемся к базе

In [2]:
path_to_db_local = r'D:\Analysis\repository_prakticum\telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Готовим данные

### SQL-запрос

Требуемые поля в готовом датасете:
- user_id;
- lt_day;
- is_new. Поле хранит информацию о том, является ли клиент новым (если количество дней «жизни» составляет не более 365 дней);
- age;
- gender_segment. Для удобства работы с полем (пол) преобразуем значения в текстовый вид;
- os_name;
- cpe_type_name;
- country;
- city;
- age_segment;
- traffic_segment;
- lifetime_segment;
- nps_score; 
- nps_group. Поле хранит информацию о том, к какой группе относится оценка клиента в опросе (критики, нейтралы, сторонники).Поскольку данные находятся в разных таблицах, объединим их.


Требуемые поля в готовом датасете:
- user_id;
- lt_day;
- is_new. Поле хранит информацию о том, является ли клиент новым (если количество дней «жизни» составляет не более 365 дней);
- age;
- gender_segment. Для удобства работы с полем (пол) преобразуем значения в текстовый вид;
- os_name;
- cpe_type_name;
- country;
- city;
- age_segment;
- traffic_segment;
- lifetime_segment;
- nps_score; 
- nps_group. Поле хранит информацию о том, к какой группе относится оценка клиента в опросе (критики, нейтралы, сторонники).

Поскольку данные находятся в разных таблицах, объединим их.ента в опросе

In [3]:
query = """
SELECT u.user_id,
       u.lt_day, 
        CASE
            WHEN u.lt_day <=365 THEN 'новый'
            ELSE 'постоянный'
        END AS is_new,
        u.age,
        CASE
            WHEN u.gender_segment = 1 THEN 'женщины'
            WHEN u.gender_segment = 0 THEN 'мужчины'
            ELSE 'неопределенный'
        END AS gender_segment,
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        a.title AS age_segment,
        tr.title AS traffic_segment,
        lt.title AS lifetime_segment,
        u.nps_score,
        CASE
            WHEN u.nps_score <= 6 THEN 'критики'
            WHEN u.nps_score > 6 AND u.nps_score <= 8 THEN 'нейтралы'
            WHEN u.nps_score > 8 THEN 'сторонники'
        END AS nps_group
FROM user AS u
LEFT OUTER JOIN location AS l ON u.location_id = l.location_id
LEFT OUTER JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
LEFT OUTER JOIN traffic_segment AS tr ON u.tr_gr_id = tr.tr_gr_id
LEFT OUTER JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id;
"""

### Создаем датафрейм по данным запроса

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,постоянный,45.0,женщины,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,постоянный,53.0,мужчины,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,постоянный,57.0,мужчины,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники


### Сохраняем данные в csv-файл

In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)